# Model selection

**Goal**: Given a reasonable feature list, try assessing different models to figure out which model family might be most successful in fitting this data. The intent is to focus on high-level architectural decisions (e..g overall structure of model)

**Output**: A report with metrics to inform which model types are likely to perform better.

**TODO**:
- 

In [16]:

# Import needed libraries
from sklearn.model_selection import StratifiedShuffleSplit

import scanpy as sc
import numpy as np
import pandas as pd
import os
import pathlib

import matplotlib.cm as cm
import matplotlib.pyplot as plt

from utils.config import *
from utils.analysis_variables import *
from utils.analysis_functions import *

import pickle

from sklearn.feature_selection import SelectKBest, GenericUnivariateSelect, f_classif
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, confusion_matrix


from sklearn.model_selection import GridSearchCV, train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_classification
from sklearn.metrics import make_scorer, confusion_matrix

import pickle


In [17]:
# Scanpy setup
sc.settings.verbosity = 3 # corresponds to hints

# Notebook setup
np.random.seed(15)

import warnings
warnings.filterwarnings('ignore')

In [18]:
# Important paths
notebook_name = "05a_model_selection"

# path_outdir_base = "../../output/20240221_import"
path_results = os.path.join(path_outdir_base, notebook_name)
os.makedirs(path_results, exist_ok=True)

path_input_data = os.path.join(path_outdir_base, "03_create_test_train", "training_data.pkl")
path_input_adata = os.path.join(path_outdir_base, "03_create_test_train", "adata_labeled.h5ad")
path_input_features = os.path.join(path_outdir_base, "04_feature_selection", "dict_feature_data.pkl")

# Import data

In [19]:
with open(path_input_data, 'rb') as f:
    data_dict = pickle.load(f)

X_train = data_dict['X']
y_train = data_dict['Y']

In [20]:
adata_labeled = sc.read_h5ad(path_input_adata)

In [21]:
# open sexually dimorphic genes
with open(os.path.join(path_outdir_base, '01_data_import_and_filtering', 'all_sex_dimorph_genes.pkl'), 'rb') as file:
    all_sex_dimorph_genes = pickle.load(file)

In [22]:
adata_labeled.var

,gene_ids,msigdb_GOBP_SEX_DIFFERENTIATION.v2023.2.Mm,msigdb_GOCC_X_CHROMOSOME.v2023.2.Mm,msigdb_GOCC_SEX_CHROMOSOME.v2023.2.Mm,msigdb_GOBP_MALE_SEX_DETERMINATION.v2023.2.Mm,msigdb_GOBP_FEMALE_SEX_DIFFERENTIATION.v2023.2.Mm,gene_mt,gene_hsp,gene_ribo,gene_hemo,n_cells_by_counts,mean_counts,pct_dropout_by_counts,total_counts,n_cells,highly_variable,means,dispersions,dispersions_norm
Xkr4,ENSMUSG00000051951,False,False,False,False,False,False,False,False,False,226,0.003229,99.682696,230.0,226,False,0.009838,1.371536,-0.003350
Gm1992,ENSMUSG00000089699,False,False,False,False,False,False,False,False,False,4,0.000056,99.994384,4.0,4,False,0.000277,1.629869,0.578111
Gm37381,ENSMUSG00000102343,False,False,False,False,False,False,False,False,False,4,0.000056,99.994384,4.0,4,False,0.000156,1.095813,-0.623953
Rp1,ENSMUSG00000025900,False,False,False,False,False,False,False,False,False,12,0.000183,99.983152,13.0,12,False,0.000266,0.851913,-1.172929
Sox17,ENSMUSG00000025902,False,False,False,False,False,False,False,False,False,497,0.055556,99.302211,3957.0,497,True,0.060993,2.812092,3.239084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Gm10931,ENSMUSG00000094350,False,False,False,False,False,False,False,False,False,8,0.000112,99.988768,8.0,8,False,0.000279,0.985874,-0.871406
AC168977.1,ENSMUSG00000079808,False,False,False,False,False,False,False,False,False,68,0.000955,99.904528,68.0,68,False,0.003081,1.317975,-0.123906
PISD,ENSMUSG00000095041,False,False,False,False,False,False,False,False,False,34475,0.859361,51.597052,61208.0,34475,True,1.309675,1.731082,0.986408
DHRSX,ENSMUSG00000063897,False,False,False,False,False,False,False,False,False,14089,0.237459,80.219024,16913.0,14089,False,0.527060,1.308353,-0.294325


In [23]:
# Import possible features

with open(path_input_features, 'rb') as file:
    dict_feature_data = pickle.load(file)

# Add all sexually dimorphic gene as a geneset for comparison
adata_labeled.var['msigdb_all_sex_dimorph_genes'] = adata_labeled.var.index.map(lambda x: True if x in all_sex_dimorph_genes else False)
dict_feature_data.update({'Sexually Dimorphic Genes': adata_labeled.var.index[adata_labeled.var.msigdb_all_sex_dimorph_genes]})

# prelim_model_features = dict_feature_data["FPR (All)"]
# adata_model = adata_labeled[:, adata_labeled.var.index.isin(prelim_model_features)]

In [24]:
# Create a hyperparameter that is the feature set to use
X_train_featurefilter = {}

for feature_set in dict_feature_data.keys():
    mask = adata_labeled.var.index.isin(dict_feature_data[feature_set])

    X_train_featurefilter.update({feature_set: X_train[:,mask]} )
    

In [25]:
# # Subset training data to only include X_data corresponding to features we want to use
# mask_features = adata_labeled.var.index.isin(prelim_model_features)
# X_train_features = X_train[:, mask_features]

# print(f"Shape of X_train: {X_train_features.shape}, Shape of Y_train: {y_train.shape}")

# Pipeline + GridSearchCV to compare performance of different types of models

## Set up search

In [26]:
# Define scoring metrics

scoring = {
    'accuracy': 'accuracy',  # Default scoring for classification
    'specificity': make_scorer(specificity),
    'false_positive_rate': make_scorer(false_positive_rate, greater_is_better=False),  # Minimize false positive rate
    'false_negative_rate': make_scorer(false_negative_rate, greater_is_better=False),  # Minimize false negative rate; something funny abt this processing makes this neg
    'precision': make_scorer(precision)
}

# Define pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', None), # classifier will be replaced during grid search
])


# Define hyperparameters to search
# param_grid = {
#     'classifier': [RandomForestClassifier(), SVC(), LogisticRegression(), GradientBoostingClassifier()],
#     'classifier__n_estimators': [50, 100], # specific to random forest; represents number of
#     'classifier__C': [1, 10], # specific fo SVC; controls regularization strength
# }

param_grid_rf = {
    'classifier': [RandomForestClassifier()],
    'classifier__n_estimators': [50, 100],
    'classifier__max_features': ["sqrt", "log2"],
}

param_grid_svc = {
    'classifier': [SVC()],
    'classifier__C': [1, 5],  # Regularization parameter
}

param_grid_lr = {
    'classifier': [LogisticRegression()],
    'classifier__C': [1, 5],  # Regularization parameter
}

param_grid_gb = {
    'classifier': [GradientBoostingClassifier()],
    'classifier__n_estimators': [50, 100],
    'classifier__learning_rate': [0.5], 
    'classifier__max_features': ["sqrt", "log2"], 
}

# Combine the param grids into a list
all_param_grids = [param_grid_rf, param_grid_svc, param_grid_lr, param_grid_gb]


# classifiers = {
#     'GradientBoosting': (GradientBoostingClassifier(), {'n_estimators': [100, 150, 200], 'learning_rate': [0.5, 0.9], 'max_features': ['sqrt']}),
#     'SVM': (SVC(), {'C': [1, 5, 10, 15], 'kernel': ['rbf', 'linear']}),
#     'LogisticRegression': (LogisticRegression(), {'C': [1, 5, 10, 15]}),
# }


## Do grid search CV for different parameter sets

In [27]:
def GridSearchCV_on_featureset(X_train, featureset_id, all_param_grids=all_param_grids, pipeline=pipeline, scoring=scoring, path_results=path_results):
    print(f"RandomizedSearchCV on {featureset_id}")
    
    # fit the model and assess
    best_params_list = []
    best_estimator_list = []
    cv_result_list = []
    cv_result_list_df = []
    cv_result_list_filtered = []

    for param_grid in all_param_grids:
        grid_search = RandomizedSearchCV(pipeline, param_grid, cv=5, scoring=scoring, refit='accuracy') #GridSearchCV
        grid_search.fit(X_train, y_train)

        # Access the best parameters and best estimator for each classifier
        best_params_list.append( grid_search.best_params_ )
        best_estimator_list.append(grid_search.best_estimator_)

        cv_results = grid_search.cv_results_
        cv_result_list.append(cv_results)

        print(cv_results['mean_test_accuracy'])

        print(f"Best Parameters: {grid_search.best_params_ }")
        print(f"Best Estimator: {grid_search.best_estimator_}")


    for cv_scores in cv_result_list:
        cv_df = pd.DataFrame.from_dict(cv_scores)

        cols_params = [x for x in cv_df.columns if x.startswith("param")]
        cols_scores =  [x for x in cv_df.columns if x.startswith("mean_")]

        cv_result_list_df.append(cv_df)
        cv_result_list_filtered.append(cv_df[cols_params+cols_scores])

    merged_cv_df = pd.concat(cv_result_list_df, join='outer', axis=0)
    merged_cv_df_filtered = pd.concat(cv_result_list_filtered, join='outer', axis=0)

    merged_cv_df.to_csv(os.path.join(path_results, 'merged_cv_df_' + featureset_id+'.csv'))
    merged_cv_df_filtered.to_csv(os.path.join(path_results, 'merged_cv_df_filtered_' + featureset_id + '.csv'))

    return(merged_cv_df)


In [28]:
# 206 minutes to run; about 100 of those on the signif feature one
results_store = []
for feature_set_id, X_train_filtered in X_train_featurefilter.items():
    result = GridSearchCV_on_featureset(X_train_filtered, feature_set_id)
    results_store.append(result)

RandomizedSearchCV on Highly Variable Genes


[0.78426001 0.803025   0.71135781 0.72337243]
Best Parameters: {'classifier__n_estimators': 100, 'classifier__max_features': 'sqrt', 'classifier': RandomForestClassifier()}
Best Estimator: Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier', RandomForestClassifier())])
[0.87511803 0.87376841]
Best Parameters: {'classifier__C': 1, 'classifier': SVC()}
Best Estimator: Pipeline(steps=[('scaler', StandardScaler()), ('classifier', SVC(C=1))])
[0.8494668  0.84933212]
Best Parameters: {'classifier__C': 1, 'classifier': LogisticRegression()}
Best Estimator: Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier', LogisticRegression(C=1))])
[0.82125028 0.84514594 0.74335109 0.8126092 ]
Best Parameters: {'classifier__n_estimators': 100, 'classifier__max_features': 'sqrt', 'classifier__learning_rate': 0.5, 'classifier': GradientBoostingClassifier()}
Best Estimator: Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier',
            

In [29]:

# # fit the model and assess
# best_params_list = []
# best_estimator_list = []
# cv_result_list = []

# for param_grid in all_param_grids:
#     grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring=scoring, refit='accuracy')
#     grid_search.fit(X_train, y_train)

#     # Access the best parameters and best estimator for each classifier
#     best_params_list.append( grid_search.best_params_ )
#     best_estimator_list.append(grid_search.best_estimator_)

#     cv_results = grid_search.cv_results_
#     cv_result_list.append(cv_results)

#     print(cv_results['mean_test_accuracy'])

#     print(f"Best Parameters: {grid_search.best_params_ }")
#     print(f"Best Estimator: {grid_search.best_estimator_}")

In [30]:
# cv_result_list_filtered = []
# for cv_scores in cv_result_list:
#     print(cv_scores['params'])
#     cv_df = pd.DataFrame.from_dict(cv_scores)

#     cols_params = [x for x in cv_df.columns if x.startswith("param")]
#     cols_scores =  [x for x in cv_df.columns if x.startswith("mean_")]

#     cv_result_list_filtered.append(cv_df[cols_params+cols_scores])


# Explore featureset performance !! TODO

In [ ]:
# how well do different featuresets do given the same architecure? 

# Save relevant data

In [33]:

with open(os.path.join(path_results, 'results_store.pkl'), 'wb') as file:
    pickle.dump(results_store, file)

NameError: name 'X_train_features' is not defined